## PPG Signal Quality 

In [2]:
# Initialize the package
import sys
import rndSignal
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

We can generate signals using the neurokit simulate functions.  
Refer generating of biosignals here: https://neurokit2.readthedocs.io/en/latest/

In [3]:
# Generate 25 second ppg signal using neurokit built-in function
import neurokit2 as nk

ppg_signal_1 = nk.ppg_simulate(
            duration = 25,
            sampling_rate = 1000,
            heart_rate = 60
        )
# Clean signal but with outlier
ppg_signal_2 = nk.ppg_simulate(
            duration = 25,
            sampling_rate = 1000,
            heart_rate = 60            
        ) 
ppg_signal_2[10000:10200] =  10
ppg_signal_2[20000:20100] =  8

# Plot the ppg signals
rndSignal.plot_signal(
    [ppg_signal_1,ppg_signal_2],
    [1000,1000], 
    labels = ["clean_ppg", "noisy_ppg"],
    x_axis_label = "Time (seconds)",
    y_axis_label = "Raw PPG Data",
    grid_plot = True, 
    grid_lines = 2,
    grid_columns = 1
)

### Signal Quality Detection 

Methods include   
a.` DeepBeat` - a deep learning method which outputs categorical results (Excellent, Acceptable, Not Acceptable)  
b. `Matching method` (continous from 0 -1) adapted from paper of (Li, H. & Huang, S. (2020)) - creates a template from the signal and compare to each beat.   
c. `SNR` - calculates the signal to noise ratio of the signal - _Not yet included in the function_    
d. `sSQI` - calculates the skewness of the input signal (According to Elgendi (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5597264/) this outperforms the gold standard (perfusion index) in calculating the SQI of the PPG signal) -_Not yet included in the function_  

In [4]:
# A. Using `deepbeat` (note: This require input signal = 25 seconds)
deepbeat_sig_1 = rndSignal.ppg_signal_quality(ppg_signal_1, 1000, method = "deepbeat")
deepbeat_sig_2 = rndSignal.ppg_signal_quality(ppg_signal_2, 1000, method = "deepbeat")
print(f"DeepBeat SQI for ppg_signal_1 = {deepbeat_sig_1}")
print(f"DeepBeat SQI for ppg_signal_2 = {deepbeat_sig_2}")

# B. Using matching method (Li, H. & Huang, S. (2020)) This require input signal > 10 seconds
template_sig_1 = rndSignal.ppg_signal_quality(ppg_signal_1, 1000, method = "template_matching")
template_sig_2 = rndSignal.ppg_signal_quality(ppg_signal_2, 1000, method = "template_matching")
print(f"Matching Method  SQI for ppg_signal_1 = {template_sig_1[0]}")
print(f"Matching Method  SQI for ppg_signal_2 = {template_sig_2[0]}")

DeepBeat SQI for ppg_signal_1 = Excellent
DeepBeat SQI for ppg_signal_2 = Not Acceptable
Matching Method  SQI for ppg_signal_1 = 0.9938679304279213
Matching Method  SQI for ppg_signal_2 = 0.9770194259295097


It can be observed that _DeepBeat_ method is sensitive to outliers and may result to outright poor quality detection. In this case we need to preprocess the signals first to remove anomalies/outliers.

### Signal Quality Detection using `ppgSignal()`
**`ppgSignal()`** has a built-in signal quality detection of the ppg beats using a random forest classifier from the calculated features of the beat. By calling the .`beats_df` of the output module we can see the beats attributes and features as well as the quality of the beats. 


In [5]:
# Process the signal using ppgSignal() - this introduces filtering
# It is important to set predict_beats = True to summarize the beats attributes
savvyppg_signal_1 = rndSignal.ppgSignal(ppg_signal_1, 1000, predict_beats=True)
savvyppg_signal_2 = rndSignal.ppgSignal(ppg_signal_2, 1000, predict_beats=True)

The ppg beats information is saved in `beats_df`. To visualize the quality of the signal, we use `ppg_plot_quality` from `plotting.ppg_plotting`

In [6]:
# Visualize the signal quality
print("ppg_signal_1")
rndSignal.ppg_plot_quality(ppg_signal_1,savvyppg_signal_1.beats_df,savvyppg_signal_1.fs)
print("ppg_signal_2")
rndSignal.ppg_plot_quality(ppg_signal_2,savvyppg_signal_2.beats_df,savvyppg_signal_2.fs)

ppg_signal_1


ppg_signal_2


Red area shows the poor beats while the green area shows the good beats.